<a href="https://colab.research.google.com/github/KhinSu2Hlaing/Book-Recommendation-System_3k1c/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
import pickle

In [ ]:
#Loading Dataset
book = pd.read_csv('/content/Books.csv')
user = pd.read_csv('/content/Users.csv')
rating = pd.read_csv('/content/Ratings.csv')

<ipython-input-8-b231f86bdc8e>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/Books.csv')


In [ ]:
# Exploratory Data Analysis
book.tail()
user.tail()
rating.tail()
print(book.shape)
print(user.shape)
print(rating.shape)

book.isnull().sum()
user.isnull().sum()
rating.isnull().sum()
book.duplicated().sum()
user.duplicated().sum()
rating.duplicated().sum()
user.info()
book.info()
rating.info()

(271360, 8)
(278858, 3)
(1149780, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          27135

In [ ]:
# Merging rating and book dataframes
rating_with_name = rating.merge(book, on='ISBN')

# Calculating the number of ratings and average rating for each book
num_rating_df = rating_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating': 'Num_rating'}, inplace=True)

avg_rating_df = rating_with_name.groupby('Book-Title').mean(numeric_only=True)['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'Avg_rating'}, inplace=True)

# Merging popularity dataframes and filtering top 100 popular books
popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')
pbr_df = popular_df[popular_df['Num_rating'] >= 300].sort_values('Avg_rating', ascending=False).head(100)

# Selecting relevant columns for recommendation
pbr_df = pbr_df.merge(book, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-M', 'Num_rating', 'Avg_rating']]

# Saving the popularity-based recommendation data into a pickle file
pickle.dump(pbr_df, open('PopularBookRecommendation.pkl', 'wb'))

In [ ]:
# Filtering users who have given ratings to at least 250 books
b = rating_with_name.groupby('User-ID').count()['Book-Rating'] > 250
users_with_ratings = b[b].index
filtered_rating = rating_with_name[rating_with_name['User-ID'].isin(users_with_ratings)]

# Filtering books that have ratings from at least 50 users
c = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = c[c].index
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

# Creating a pivot table for collaborative filtering
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace=True)

# Calculating cosine similarity for collaborative filtering
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)

# Function to recommend similar books
def recommendation(book_name):
    # Fetching index
    index = np.where(np.array(list(pt.index)) == book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), reverse=True, key=lambda x: x[1])[1:9]

    data = []
    for i in similar_items:
        item = []
        temp_df = book[book['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(item)
    return data

# Saving collaborative filtering data into pickle files
pickle.dump(pt, open('pt.pkl', 'wb'))
pickle.dump(book, open('book.pkl', 'wb'))
pickle.dump(similarity_scores, open('similarity_scores.pkl', 'wb'))